In [139]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [140]:
df = pd.DataFrame(
    {
        "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
        "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
        "C": np.random.randn(8),
        "D": np.random.randn(8),
    }
)

df

,A,B,C,D
0,foo,one,1.268824,0.404128
1,bar,one,1.279036,-1.581991
2,foo,two,1.094762,0.762019
3,bar,three,-0.125674,-1.914041
4,foo,two,-0.064364,-0.172677
5,bar,two,1.510112,1.095183
6,foo,one,-0.706736,0.167282
7,foo,three,0.908649,1.964041


On a DataFrame, we obtain a GroupBy object by calling **groupby()**. This method returns a pandas.api.typing.DataFrameGroupBy instance. We could naturally group by either the A or B columns, or both.

Collectively we refer to the grouping objects as the **keys**

In [141]:
grouped = df.groupby("A")  # df.groupby('A') is just syntactic sugar for df.groupby(df['A']).
grouped

In [142]:
df2 = df.set_index(["A", "B"])  # set multi index
df2

C         D
A   B                        
foo one    1.268824  0.404128
bar one    1.279036 -1.581991
foo two    1.094762  0.762019
bar three -0.125674 -1.914041
foo two   -0.064364 -0.172677
bar two    1.510112  1.095183
foo one   -0.706736  0.167282
    three  0.908649  1.964041

In [143]:
grouped = df2.groupby(level=df2.index.names.difference(["B"]))
grouped.sum()

,C,D
A,,
bar,2.663474,-2.400848
foo,2.501134,3.124793


In [144]:
C = np.arange(1, 7, dtype=float)
C[[3, 5]] = np.nan
df = pd.DataFrame({
    "A" : [1, 1, 1, 2, 2, 2],
    "B" : [1, 1, 2, 2, 1, 1],
    "C": C,
})
df

,A,B,C
0,1,1,1.0
1,1,1,2.0
2,1,2,3.0
3,2,2,NaN
4,2,1,5.0
5,2,1,NaN


In [145]:
gbB = df.groupby("B")
gbB

In [146]:
gbB.get_group(2)  # Filters the rows such that only the rows where B = 2 is shown.

,A,B,C
2,1,2,3.0
3,2,2,NaN


In [147]:
gbB.get_group(1)  # Filters the rows such that only the rows where B = 1 is shown. Notice the groupby object keeps the original indexes.

,A,B,C
0,1,1,1.0
1,1,1,2.0
4,2,1,5.0
5,2,1,NaN


In [148]:
gbB.sum()  # Grouped by the different values in B, each group is summed. There are 2 groups: B = 1 group and B = 2 group. Notice B column is now index:

,A,C
B,,
1,6,8.0
2,3,3.0


In [149]:
# let's get back to original df
df

,A,B,C
0,1,1,1.0
1,1,1,2.0
2,1,2,3.0
3,2,2,NaN
4,2,1,5.0
5,2,1,NaN


In [150]:
gbA = df.groupby("A")
gbA.sum()  # Notice in the A=1 group, sum of C values is as we expect.
           # However in the A=2 group, NaN values are just dropped and only 5.0 is summed.

,B,C
A,,
1,4,6.0
2,4,5.0


In [151]:
gbA.count()  # count method also shows the same thing.

,B,C
A,,
1,3,3
2,3,1


In [152]:
# method 1
gbA.agg(np.sum)

,B,C
A,,
1,4,6.0
2,4,5.0


In [153]:
# method 2
gbA.aggregate(sum)

,B,C
A,,
1,4,6.0
2,4,5.0


In [154]:
# method 3
gbA.corr()  # A correlation matrix is shown where values of B and C are grouped into 2 different groups (A=1 and A=2)
            # And then 2 separate correlation matrixes are shown with values of A as indexes
            # Notice this is a multi-index dataframe.

B         C
A                      
1 B  1.000000  0.866025
  C  0.866025  1.000000
2 B  1.000000       NaN
  C       NaN       NaN

In [155]:
type(gbA.corr())  # Notice after using a method on groupby object, the type is DataFrame.

pandas.core.frame.DataFrame

In [156]:
corrdf = gbA.corr()
corrdf.iloc[1, :]  # Second row and all columns. Notice the B and C in the index is not shown.

B    0.866025
C    1.000000
Name: (1, C), dtype: float64

In [157]:
corrdf.iloc[:, 0]  # All rows and first column.

A   
1  B    1.000000
   C    0.866025
2  B    1.000000
   C         NaN
Name: B, dtype: float64

In [158]:
corrdf.loc[:, "B"]  # same as above, only with .loc() method.

A   
1  B    1.000000
   C    0.866025
2  B    1.000000
   C         NaN
Name: B, dtype: float64

In [159]:
# Because this is a multi-index dataframe, we can access individual elements like this:
corrdf.loc[1, :]["B"]  # Select the rows where A=1. Then, inside that select the rows where the 2nd index is "B"

B    1.000000
C    0.866025
Name: B, dtype: float64

In [160]:
# Now group by both A and B
gbAB = df.groupby(["A", "B"])

The index of the combined DataFrame will be the group identifiers, with one index level per group key.

In [161]:
gbAB.count()

C
A B   
1 1  2
  2  1
2 1  1
  2  0

In [163]:
df.isnull()

,A,B,C
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,True
4,False,False,False
5,False,False,True


In [165]:
gbC = df.isnull().groupby("C")
gbC.sum()  # False is equal to 0, True is equal to 1 in numeric operations.

,A,B
C,,
False,0,0
True,0,0


In [162]:
def num_missing(df):
    "Return the number of missing items in each column of df"
    return df.isnull().sum()

In [166]:
num_missing(df)

A    0
B    0
C    2
dtype: int64

In [167]:
gbA.agg(num_missing)  # now the dataframe is grouped into 2 groups depending on the value of A column.

,B,C
A,,
1,0,0
2,0,2


The key to keep in mind is that the function we pass to agg should take in a DataFrame (or Series) and return a Series (or single value) with one item per column in the original DataFrame.

* When the function is called, the data for each group will be passed to our function as a DataFrame (or Series).

In [168]:
def smallest_by_b(df):
    return df.nsmallest(2, "B")

In [169]:
gbA.apply(smallest_by_b)

A  B    C
A             
1 0  1  1  1.0
  1  1  1  2.0
2 4  2  1  5.0
  5  2  1  NaN

Notice that:

* The first (outer) index is the group level
* The second (inner) index is the original index

